<a href="https://colab.research.google.com/github/RajSoumyadip/Car-Classification/blob/master/TransferLearningResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/RajSoumyadip/Car-Classification.git

Cloning into 'Car-Classification'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 483 (delta 3), reused 4 (delta 0), pack-reused 472
Receiving objects: 100% (483/483), 49.97 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
PATH = './Car-Classification/dataset/'
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step


In [3]:
import os
classes = []
for d in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, d)) and not d.startswith('.'):
        classes.append(d) 
print ("There are ", len(classes), "classes:\n", classes)



There are  3 classes:
 ['Hyundai', 'Mercedes', 'Honda']


In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
data  = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=300, bs=16, valid_pct=0.1).normalize(imagenet_stats)

In [5]:
print ("There are", len(data.train_ds), "training images and", len(data.valid_ds), "validation images." )

There are 416 training images and 46 validation images.


In [6]:
from keras.preprocessing.image import ImageDataGenerator
TRAIN_DIR = "./Car-Classification/dataset"
BATCH_SIZE = 16
train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 462 images belonging to 3 classes.


In [7]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["Honda", "Hyundai", "Mercedes"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
def plot_training(history):
    acc = history.history['acc']
    #val_acc = history.history['val_acc']
    loss = history.history['loss']
    #val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    #plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')

In [16]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 10
BATCH_SIZE = 16
num_train_images = 10000

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="./Car-Classification/checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list,)





Epoch 1/10
625/625 [==============================] - 139s 223ms/step - loss: 0.3572 - acc: 0.8801

Epoch 00001: saving model to ./Car-Classification/checkpoints/ResNet50_model_weights.h5
Epoch 2/10
625/625 [==============================] - 132s 211ms/step - loss: 0.2242 - acc: 0.9166

Epoch 00002: saving model to ./Car-Classification/checkpoints/ResNet50_model_weights.h5
Epoch 3/10
625/625 [==============================] - 134s 214ms/step - loss: 0.1663 - acc: 0.9390

Epoch 00003: saving model to ./Car-Classification/checkpoints/ResNet50_model_weights.h5
Epoch 4/10
625/625 [==============================] - 132s 210ms/step - loss: 0.1198 - acc: 0.9551

Epoch 00004: saving model to ./Car-Classification/checkpoints/ResNet50_model_weights.h5
Epoch 5/10
625/625 [==============================] - 132s 212ms/step - loss: 0.1062 - acc: 0.9639

Epoch 00005: saving model to ./Car-Classification/checkpoints/ResNet50_model_weights.h5
Epoch 6/10
625/625 [==============================] - 134s 2

In [0]:
history

AttributeError: ignored